# Traiter un corpus en language naturel

Après cette rapide présentation de quelques bases du language, on va pouvoir attaquer les choses sérieuses: travailler sur du vrai texte pour faire un traitement "en masse": c'est de la **lecture distante**. L'idée de la lecture distante, c'est d'arriver à produire une interprétation d'un texte (ou corpus) sans avoir à le lire, avec des méthodes computationnelles.

On va commencer dans ce notebook par quelque chose de très simple:
- lire notre fichier texte contenant le corpus
- le simplifier rapidement pour faciliter son traitement
- établir une liste de tous les mots que contient le texte, et faire compter le nombre d'occurrences de chaque mot.

---

## Lire les contenus d'un fichier

Pour lire un fichier, on lance la commande:

In [ ]:
with open("in_correspondance_matsutaka_full.txt", mode="r", encoding="utf-8") as fichier: 
    corpus = fichier.read()
print(corpus)  # afficher les contenus du fichier

Avec ce bloc de code, on voit quelques subtilités supplémentaires de Python. 


`with open(...) as fichier:` permet d'ouvrir un fichier. Si on décortique, il y a ici 3 nouveautés: 
- la fonction `open()` pour ouvrir un fichier
- la syntaxe `with...as`
- le rôle de l'indentation.

### `open()`, c'est la fonction Python qu'on utilise pour ouvrir le fichier.

Ici, on lui donne 2 arguments:
- en premier, le nom du fichier `in_correspondance_matsutaka_full.txt`
- en deuxième, un argument nommé `mode` auquel on donne la valeur `"r"`. `mode` permet d'indiquer pour quoi on ouvre le fichier:
    - `mode="r"` = on ouvre le fichier en lecture
    - `mode="w"` = on ouvre le fichier en écriture. Attention, `mode="w"` **efface tous les contenus du fichier qu'on ouvre !**.


`open` ouvre le fichier, mais n'affiche pas ses contenus. Pour afficher les contenus, il faut utiliser `fichier.read()` : on appelle la *méthode* `read` sur la variable `fichier`, et donc on affiche ce qui est contenu.
- si on `print` juste notre variable `fichier`, on voit que ce qui s'affiche n'est pas le contenu du fichier, mais un object `TextWrapper`.
- par analogie, c'est comme quand on reçoit une lettre (ou un colis Vinted): `fichier`, c'est l'emballage, et on doit prendre l'emballage et l'ouvrir pour voir son contenu avec `fichier.read()`

In [ ]:
with open("in_correspondance_matsutaka_full.txt", mode="r", encoding="utf-8") as fichier:
    print(fichier)

### La syntaxe `with ... as`

Pour faire bref, la syntaxe `with open(...) as fichier` permet d'assigner le résultat de la fonction `open` à la variable `fichier`. C'est donc une variante du code en dessous:

In [ ]:
fichier = open("in_correspondance_matsutaka_full.txt", mode="r", encoding="utf-8")
fichier.close()  # si on utilise pas with...as, il faut explicitement fermer le fichier

On préfère `with ... as` quand on travaille avec des fichiers car ça permet de fermer un fichier automatiquement, sans avoir à utiliser `fichier.close()` (et il faut éviter de garder des fichiers ouverts inutilement en Python !)

### Indentation

Si votre regard est assez acéré, vous aurez remarqué que après `with ... as ... :` on a un retour à la ligne suivi d'une indentation (alinéa après un retour à la ligne). Pourquoi ? 

En Python, ***l'indentation est signifiante et permet de structurer le code en blocs*** ! 

Le `:` et l'indentation servent de délimiteurs entre différents blocs de code. Cette indentation doit être utilisée de manière cohérente dans tout votre code. La convention consiste à utiliser 4 espaces comme indentation (certains environnements de développement transforment automatiquement une tabulation en 4 espaces). Cela signifie qu'après avoir utilisé un deux-points (comme dans notre `with...as`), la ligne suivante devrait être indentée de quatre espaces de plus que la ligne précédente.

Concrètement, dans le bloc de code ci-dessous, le contenu de la variable `fichier`n'est accessible que dans le bloc indenté après le `with...as`:

In [ ]:
with open("in_correspondance_matsutaka_full.txt", mode="r", encoding="utf-8") as fichier:
    # ici, on est dans le bloc, indenté, `fichier.read()` est possible
    corpus = fichier.read()
    print(corpus[:100])  # [:100] permet d'afficher les 100 premiers caractères de `corpus`

# ici, on est sorti du bloc, donc le fichier a été fermé. 
# `fichier.read()` nous envoie une belle erreur, c'est normal !
print(fichier.read())

### Pour résumer

On a vu ici comment ouvrir un fichier avec la fonction `open` et lire ses contenus avec `read()`. En plus de ça, on a vu la construction `with ... as`, et surtout, le rôle de l'indentation pour structurer son code en blocs.

---

## Le corpus

Avant de se lancer dans l'analyse, on regarde ce que contient notre texte. Pour rappel, la variable `corpus` contient les contenus du fichier. 

In [ ]:
# Exercice: afficher ici les contenus du fichier.

In [ ]:
# la fonction `type` permet de connaître le type d'une valeur
print(type(corpus))

Le corpus utilisé pour cet atelier est un ensemble de correspondances autour de l'achat par Matsukata d'un ensembles d'œuvres européennes, notamment par l'intermédiaire du conservateur de musées Léonce Bénédite, durant la première moitié du XXe siècle. Les originaux sont conservés dans les archives de l'Institut national d'histoire de l'art et du Musée Rodin. La version numérique du corpus a été produite par Léa Saint-Raymond.

Comme on le voit, la structure du texte est toujours la même, ce qui est bien pratique pour nous: la première étape de la fouille de texte, c'est de structurer ses données. En effet, en lisant le texte on identifie une structure qui se répète. L'ordinateur, par contre, lit le texte de manière "bête et méchante" : caractère par caractère. Par exemple, il ne peut même pas faire la différence entre les différentes lettres de la correspondance.

Il va donc falloir transformer notre document pour le rendre sa structure compréhensible par un ordinateur. Une fois que l'ordinateur peut comprendre la structure, alors il peut identifier les différentes lettres de la correspondance, les mots, les phrases... et donc réaliser des opérations sur ces éléments. 

**Analyser le texte, c'est donc le structurer pour le rendre compréhensible par ordinateur**. Le type de données du texte (`str`) n'est par nature pas structuré, donc ne permet pas par lui-même cette analyse.

---

## Combien de lettres dans le corpus ?

Une première manière d'aborder le corpus, c'est: **combien de lettres contient le corpus** ?

Dans le notebook précédent, on a vu la fonction `len()`:

In [ ]:
print(len(corpus))  # à quoi correspond ce résultat ?

On affiche bien la longueur de `corpus`, mais pas le nombre de lettres. Ce qu'il faudrait, ce serait séparer les lettres entre elles. Et pour ça, le type `str` offre la méthode `split()`

In [ ]:
print(corpus.split())

In [ ]:
print(len(corpus.split()))   # et là, à quoi correspond cette longueur ?

Vous l'aurez deviné, `split()` permet de séparer une `str` en une liste d'items. 

En faisant `corpus.split()`, on scinde le texte à chaque fois qu'il y a un espace. L'espace est le **séparateur par défaut** de la fonction `split()`. Mais on peut aussi utiliser d'autres séparateurs, que l'on indique en argument de la fonction `split()`.

En python la chaîne de caractères `\n` sert à représenter un retour à la ligne (n = abbréviation de *newline*). Si on veut scinder le texte ligne par ligne, on utilise donc:

In [ ]:
corpus.split("\n")

Pour le moment, on a séparé les différentes lignes du corpus. Mais on veut séparer les différentes lettres du corpus. Or, il existe une ligne vide entre chaque lettre. Une ligne vide correspond à 2 retours à la ligne, soit `\n\n`.

Comment adapter la fonction `split()` pour séparer le corpus lettre par lettre ?

In [ ]:
# Exercice: "splitter" `corpus` par "\n\n" pour créer une liste de lettres. 
# À partir de là, afficher le nombre de lettres.

---

## Les listes

OK, on a réussi à afficher le nombre de lettres dans notre corpus, c'est super ! Maintenant, regardons d'un peu plus près ce que produit `split()`.

In [ ]:
print(corpus)

In [ ]:
print(type(corpus))

In [ ]:
corpus_liste = corpus.split("\n\n")
print(corpus_liste)

In [ ]:
print(len(corpus_liste))
print(type(corpus_liste))

`split()` transforme notre *chaîne de caractères* `corpus` en une *liste* de lettres. Qu'est-ce que ça veut dire ?

Une chaîne de caractères (`str`) n'est pas structurée: l'ordinateur la traite lettre-par-lettre, et donc même par défaut, il ne va pas faire la différence entre des mots, des lignes, ou les différentes lettres de la correspondance. Or, si on se limite à une analyse caractère par caractères, on ne va pas aller bien loin.

Par contre, avec `split`, on a réussi à scinder `corpus` en mots, lignes, puis lettres, des unités qui sont plus signifiantes pour nous. Comment ? ***En transformant la `str` en `list`***.

### Les listes, une définition rapide 

Une liste est un type de données qui permet de **stocker une collection de valeurs**.
- elle s'écrit **entre crochets `[]`**
- les différents items de la liste sont **séparés par des virgules `,`**
- une liste peut contenir **n'importe quel type de données**: str, int, autres listes... On peut donc arriver à des structures de données assez complexes.
- une liste est **ordonnée et indexée**, c'est-à-dire que:
    - les différents items ont une position définie (leur position ne change pas, le 1e, 2e, 3e élément sont toujours à la même place)
    - on accède à un item d'une liste à partir de son `index`, c'est à dire de sa position dans la liste.

Voilà comment on définit une liste:

```python
ma_liste = ["un", "deux", "trois"]
```

### L'idexation d'une liste

Comme on l'a dit, une liste est ordonnée et on accède aux différents éléments à partir de leur position dans la liste. On va donc faire quelques manipulations pour voir comment ça marche.

In [ ]:
# on prend une phrase
phrase = "La saga Twilight, d'abord publiée en France sous le nom de Saga du désir interdit, est une série de romans fantastiques et sentimentaux de Stephenie Meyer publiés entre 2005 et 2020."
print(phrase)

# on la transforme en une liste de mots
phrase = phrase.split()
print(phrase)
print(len(phrase))  # comment interpréter `len` ici ?

In [ ]:
# à quoi accède-t-on ici, dans chaque cas?
print(phrase[0])
print(phrase[1])
print(phrase[-1])

Au dessus, on voit la syntaxe de base pour cibler un item d'une liste par son index:

```python
ma_liste[index]
```

Quelques spécificités:
- **l'indexation commence à 0**: `ma_liste[0]` renvoie le 1er élément de la liste, `ma_liste[1]` renvoie le 2e élément...
- **l'indexation négative** permet de récupérer les derniers éléments d'une liste et commence à -1: `ma_liste[-1]` permet d'obtenir le dernier élément de la liste, `ma_liste[-2]` l'avant dernier.
- on peut aussi récupérer **une tranche d'items** dans une liste: `ma_liste[2:4]` renvoie le 3e, 4e et 5e items de la liste.
    - la syntaxe, c'est: `ma_liste[debut:fin]`
 
Voilà un schéma pour résumer: 

![Schéma résumant l'indexation de liste](https://static.javatpoint.com/python/images/lists-indexing-and-splitting.png)

*Ps: une chaîne de caractère aussi est indéxée ! Mais dans ces cas, le premier item, c'est la première lettre, etc.*

Soit une chaîne de caractère `"Monty Python"`, l'indexation fonctionne comme ceci:

![Schéma résumant l'indexation d'une chaîne de caractère](https://camo.githubusercontent.com/663de286b76da0a6e181998831ca596b67146ecfb2d6589548e87bff5ca8a80e/687474703a2f2f7777772e6e6c746b2e6f72672f696d616765732f737472696e672d736c6963696e672e706e67)

In [ ]:
# Exercices: manipuler nos listes `phrase` et `corpus_liste`
# attention, l'indexation commence à 0 !

# 1) afficher le 3e item de `phrase`
print("Résultat de l'exercice 1:")

# 2) afficher entre le 3e et le 5e item de `phrase`
print("Résultat de l'exercice 2:")

# 3) afficher la 2e lettre de `corpus_liste`
print("Résultat de l'exercice 3:")

# 4) afficher l'avant dernière lettre de `corpus_liste`
print("Résultat de l'exercice 4:")

---

## Objectif final: écrire un compteur de mots

L'objectif du reste du notebook, ça va être d'écrire un compteur de mots. Ça veut dire, prendre tout le corpus et écrire un processus qui permette d'obtenir tous les mots uniques présents dans le corpus, et le nombre de fois que chaque mot apparaît. On va faire ce processus ensemble pour voir comment on peut décomposer une question simple en plusieurs opérations Python.

Ce processus comprend les étapes suivantes:
1) ouvrir le fichier contenant le corpus et le lire (déjà fait)
2) retirer les métadonnées pour ne garder que la source primaire (le contenu des lettres)
3) simplifier le corpus pour que le comptage plus qualitatif
4) écrire le compteur à proprement parler

---

## 1. Ouvrir et lire le fichier

In [ ]:
# on reprend le code écrit jusqu'à maintenant pour plus de clarté
with open("in_correspondance_matsutaka_full.txt", mode="r") as fichier: 
    corpus = fichier.read()

corpus = corpus.replace("\ufeff", "")  # pas bien important
corpus_liste = corpus.split("\n\n")

# on affiche quelques stats
print("nombre de lettres dans le corpus:", len(corpus_liste))
print("")
print("première lettre du corpus:")
print("")
print(corpus_liste[0])

---

## 2. Retirer les métadonnées du corpus

On voit ici que nos lettres ont la structure suivante:

```
Date <...>
Expéditeur <...>
Lieu d’expédition <...>
Destinataire <...>
Lieu destination <...>
<Titre donné>
<Contenu de la lettre>
```

On veut écrire un compteur de mots, mais pour le moment, on voit que notre corpus **mélange de l'information primaire (le contenu des lettres) et secondaires (leurs métadonnées)**. On ne peut pas faire l'analyse sur ces deux types d'informations à la fois: notre compteur de mots sera seulement fait sur la source primaire. Il va donc falloir supprimer toutes les premières lignes de métadonnées.

On va donc partir de `corpus_liste` et enlever les métadonnées de toutes les lettres, et stocker le résultat dans `corpus_liste_contenu`.

In [ ]:
# voir la correction dans : correction_extraction_contenu.ipynb

# supprimer les métadonnées de corpus lettres

# le processus: on va créer une liste intitulée `corpus_liste_contenu` 
# où chaque item de la liste est une lettre (type `str`), qui ne contienne pas
# les 6 premières lignes de métadonnées.

corpus_liste_contenu = []Urologue

# à nous de jouer

# À SUPPRIMER
corpus_liste_contenu = [ lettre.split("\n")[6:] for lettre in corpus_liste ]
corpus_liste_contenu = [ "\n".join(line for line in lettre) for lettre in corpus_liste_contenu ]

print(len(corpus_liste_contenu))
# print(corpus_liste_contenu[0])

In [ ]:
print(corpus_liste_contenu)

---

## Les boucles `for`

Dans le bloc de code au dessus, on a vu quelque chose de nouveau (encore !) : les boucles `for`. Les boucles, c'est un truc central en Python, et dans plein de langages de programmation. Quand on travaille sur des types complexes, comme des listes, qui peuvent être décomposés en plusieurs items, une boucle `for` permet **d'accéder à tous les items de la liste pour y appliquer une opération**.

Par analogie, c'est comme si on voulait peler une botte de carottes: 
- la botte de carotte est une liste de carottes;
- on boucle sur chaque item/carotte de la liste
- on effectue une opération: on pèle la carotte

La syntaxe est:

```python
for mon_item in ma_liste:
    print(mon_item)  # ici, je mets un print, mais on peut mettre n'importe quelle opération dans une boucle
```

todo: 
- simplifier le contenu de corpus_liste_contenu
- créer notre compteur

---

## 3. Simplifier notre corpus

`corpus_liste_contenu` contient maintenant une `list` de toutes nos lettres, sans métadonnées. Chaque lettre est une `str`. Il ne reste que la donnée primaire et c'est sur cette donnée qu'on va écrire notre compteur de mots.

Il était utile que `corpus_liste` et `corpus_liste_contenu` soit des listes puisque ça nous permettait d'accéder à chaque lettre et donc d'enlever les métadonnées des lettres. Par contre, le compteur de mots sera fait au niveau du corpus, et pas de la lettre. On transforme donc la `list` `corpus_liste_contenu` en `str`.

In [ ]:
# on se rappelle de ce que fait la méthode `join` ?
corpus_contenu = "\n\n".join(lettre for lettre in corpus_liste_contenu)

print(corpus_contenu)

In [ ]:
print(type(corpus_contenu))

On peut commencer à simplifier !

Simplifier du texte, à la base, c'est simplement faire du "chercher-remplacer" en masse. Fort heureusement, Python incorpore de chouettes fonctions pour le traitement de chaînes de caractères.
Ici, on va en utiliser deux:
- `lower()`, qui convertit le texte en minuscule
- `replace()`, qui remplace une chaîne de caractère par une autre.

In [ ]:
corpus_contenu = corpus_contenu.lower()

# quelques signes de ponctuation communs dans le texte à supprimer. 
# il y en a surement d'autres, mais bon avec ça on se sera déjà débasarré de la majorité. 
caracteres_a_supprimer = [".", ",", ";", ":", "?", "!", "•", "[", "]", "(", ")", "«", "»", "*", "_", "–"]
# qu'est-ce que je fais ici ?
for caractere in caracteres_a_supprimer:
    corpus_contenu = corpus_contenu.replace(caractere, "")

Très bien, on a maintenant notre corpus simplifié. Il n'y a maintenant plus besoin de travailler au niveau de la lettre ou du corpus, mais au niveau du mot. 

D'abord, on transforme `corpus_contenu` en une liste de tous les mots du corpus. 

Ensuite, on supprime quelques mots qui reviennent très souvent et qui biaiseront les résultats: "mon", "ma", "je" etc., et tous les mots ne contenant que 1 ou 2 lettres.

In [ ]:
corpus_mots_full = corpus_contenu.split()  # tous les mots du corpus
corpus_mots = [] # ici, on ne mettra que les mots qui 

# le corpus est en français et en anglais ! on met donc des mots des deux langues
mots_a_supprimer = [ "je", "tu", "il", "elle", "nous", "vous", "ils", "elles", 
                     "mr", "mme", "mon", "ma", "mes", "notre", "votre", "son", "sa", "des",
                     "avec", "les", "aux", "est", "que", "plus", "tout",
                     "and", "for", "with", "mine", "yours", "his", "hers", "our", "ours",
                     "their", "theirs", "the", "has", "that", "all" ]

for mot in corpus_mots_full:
    if mot not in mots_a_supprimer and len(mot) >= 3:
        corpus_mots.append(mot)

print("longueur avant nettoyage:", len(corpus_mots_full)) 
print("longueur après nettoyage:", len(corpus_mots)) 
print(corpus_mots[:15])
print(corpus_mots[-15:])

On a maintenant une liste de mots (en français et en anglais, en minuscule.

On pourrait aller plus loin dans le nettoyage: on pourrait séparer les corpus français et anglais, supprimer les espaces, retirer les noms propres... Il existe des listes de *stop words* qui contiennent tous le mots "inutiles" (mots qui servent à la structure plus qu'au contenu), et on pourrait utiliser une liste comme ça plutôt que notre liste faite main dans `mots_a_supprimer`. On ne va pas faire tout ça dans le cadre de ce cours.

---

## `if` et `else`: les conditions

# TODO

---

## 4. Écrire un compteur

Il est maintenant temps d'écrire notre fameux compteur !

In [ ]:
# on a besoin de deux choses pour notre compteur: 
# d'abord, une liste dédoublonnée de tous les mots de notre corpus.
# ensuite, un décompte du nombre d'occurrences pour chaque mot.

# expliquer tt ça
compteur = {}
mots_uniques = set(corpus_mots)

for mot in mots_uniques:
    compteur[mot] = 0

for mot in corpus_mots:
    compteur[mot] += 1

print(max(list(compteur.values())))